# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st
import numpy as np
from numpy.random import RandomState

Загрузка библиотек

In [4]:
df0 = pd.read_csv('/datasets/geo_data_0.csv')
df0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Загрузка и ознакомление с данными по первому региону. Параметры соответствуют указанным в условии задачи

In [5]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропуски отсутствуют, формат соответствует назначению данных

In [6]:
df1 = pd.read_csv('/datasets/geo_data_1.csv')
df1.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


Загрузка и ознакомление с данными по второму региону. Параметры соответствуют указанным в условии задачи

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропуски отсутствуют, формат соответствует назначению данных

In [8]:
df2 = pd.read_csv('/datasets/geo_data_2.csv')
df2.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Загрузка и ознакомление с данными по третьему региону. Параметры соответствуют указанным в условии задачи

In [9]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропуски отсутствуют, формат соответствует назначению данных

## Обучение и проверка модели

### Первый регион

In [10]:
features0 = df0.drop(['id', 'product'], axis=1)
features0.head()

,f0,f1,f2
0,0.705745,-0.497823,1.221170
1,1.334711,-0.340164,4.365080
2,1.022732,0.151990,1.419926
3,-0.032172,0.139033,2.978566
4,1.988431,0.155413,4.751769


Выделены признаки. Product является целевым признаком, а id неинформативным, то есть не влияющим на целевой. Соответственно, они удалены

In [11]:
target0 = df0['product']
target0.head()

0    105.280062
1     73.037750
2     85.265647
3    168.620776
4    154.036647
Name: product, dtype: float64

Выделен целевой признак

In [12]:
features0_train, features0_valid, target0_train, target0_valid = train_test_split(features0, target0, test_size=0.25, random_state=12345)
features0_train.shape

(75000, 3)

Выборка разделена на тестовую и валидационную в рекомендованном соотношении 3:1

In [13]:
model = LinearRegression()
model.fit(features0_train, target0_train)
predictions0_valid = model.predict(features0_valid)
mean0 = predictions0_valid.mean()
print('Средний запас предсказанного сырья:', mean0)
RMSE0 = mean_squared_error (target0_valid, predictions0_valid) ** 0.5
print('RMSE предсказания:', RMSE0)

Средний запас предсказанного сырья: 92.59256778438035
RMSE предсказания: 37.5794217150813


В соответствии с условиями задачи обучена модель линейной регрессии. Рассчитан средний запас и стандартное отклонение для первого региона 

### Второй регион
Порядок действий аналогичен первому, поэтому я не комментирую отдельно каждый шаг

In [14]:
features1 = df1.drop(['id', 'product'], axis=1)
target1 = df1['product']
features1_train, features1_valid, target1_train, target1_valid = train_test_split(features1, target1, test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features1_train, target1_train)
predictions1_valid = model.predict(features1_valid)
mean1 = predictions1_valid.mean()
print('Средний запас предсказанного сырья:', mean1)
RMSE1 = mean_squared_error (target1_valid, predictions1_valid) ** 0.5
print('RMSE предсказания:', RMSE1)

Средний запас предсказанного сырья: 68.728546895446
RMSE предсказания: 0.893099286775617


Обучена модель линейной регрессии, рассчитан средний запас и стандартное отклонение для второго региона. По сравнению с первым регионом средний запас сырья ниже примерно на четверть. Но при этом стандартная ошибка меньше почти в 50 раз.

### Третий регион
Порядок действий аналогичен предыдущим

In [15]:
features2 = df2.drop(['id', 'product'], axis=1)
target2 = df2['product']
features2_train, features2_valid, target2_train, target2_valid = train_test_split(features2, target2, test_size=0.25, random_state=12345)
model = LinearRegression()
model.fit(features2_train, target2_train)
predictions2_valid = model.predict(features2_valid)
mean2 = predictions2_valid.mean()
print('Средний запас предсказанного сырья:', mean2)
RMSE2 = mean_squared_error (target2_valid, predictions2_valid) ** 0.5
print('RMSE предсказания:', RMSE2)

Средний запас предсказанного сырья: 94.96504596800489
RMSE предсказания: 40.02970873393434


Обучена модель линейной регрессии, рассчитан средний запас и стандартное отклонение для третьего региона. Результаты близки к показателям первого. Обобщая можно отметить что первый и третий регион схожи между собой по ключевым показателям, а второй меньше по среднему ожидаемому показателю запасов, но характеризуется значительно более высокой  точностью предсказаний

## Подготовка к расчёту прибыли

In [16]:
outlay = 10000000000
margin = 450000
m_wells = 500
n_wells = 200
min_region = outlay / margin
mean_well = min_region / n_wells
print('Минимальный объем запасов в регионе для достижения точки безубыточности составляет:', min_region)
print('Средний объем запасов в скважине для достижения точки безубыточности составляет:', mean_well)

Минимальный объем запасов в регионе для достижения точки безубыточности составляет: 22222.222222222223
Средний объем запасов в скважине для достижения точки безубыточности составляет: 111.11111111111111


Добавлены следующие переменные. По условиям задачи: outlay - расходы на добычу в регионе, margin - переменный доход от тысячи баррелей, m_wells - количество скважин для исследования, n_wells - количество выбранных лучших скважин. Рассчитываемые переменные: min_region - минимальный объем сырья для достижения точки безубыточности, mean_well - средний запас в скважине с учетом их количества. Можно отметить что необходимый для достижения безубыточности средний объем запасов в 111 тысяч баррелей превышает средние предсказанные значения, которые составляют 92, 68 и 94 тб соответственно. Следовательно, разработка большей части скважин является убыточной

## Расчёт прибыли и рисков 

### Базовый расчет прибыли

In [17]:
def simple_profit(product):
    profit = product * margin - outlay 
    return profit

 Функция для расчета прибыли по скважинам

In [18]:
predictions0_sorted = pd.Series(predictions0_valid)
predictions0_sorted = predictions0_sorted.sort_values(ascending=False)
predictions0_selected = predictions0_sorted[:n_wells]
result0 = simple_profit(predictions0_selected.sum())/1000000000
display(result0)

3.99604887746513

В данной ячейке рассчитана прибыль (в миллиардах рублей) от добычи нефти по 200 лучшим скважинам в первом регионе

In [19]:
predictions1_sorted = pd.Series(predictions1_valid)
predictions1_sorted = predictions1_sorted.sort_values(ascending=False)
predictions1_selected = predictions1_sorted[:n_wells]
result1 = simple_profit(predictions1_selected.sum())/1000000000
display(result1)

2.4857120519735414

Расчитана прибыль в миллиардах рублей от добычи нефти по 200 лучшим скважинам во втором регионе

In [20]:
predictions2_sorted = pd.Series(predictions2_valid)
predictions2_sorted = predictions2_sorted.sort_values(ascending=False)
predictions2_selected = predictions2_sorted[:n_wells]
result2 = simple_profit(predictions2_selected.sum())/1000000000
display(result2)

3.3217543962432554

Расчитана прибыль в миллиардах рублей от добычи нефти по 200 лучшим скважинам в третьем регионе. Можно отметить что наиболее перспективным является первый регион, далее идет третий и замыкает тройку второй регион

### Оценка рисков

In [21]:
def boot_profit(target, predictions):
    profit = 0
    pred_sorted = predictions.sort_values(ascending=False).head(n_wells)
    selected = target[pred_sorted.index][:n_wells]
    for well in selected:
        profit += (well-mean_well)*margin
    return profit

def bootstrap (target, predictions):
    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=m_wells, random_state=state, replace=True)
        probs_subsample = (pd.Series(predictions, index=target.index))[target_subsample.index]
        values.append(boot_profit(target_subsample, probs_subsample))
    values = pd.Series(values)
    lower = values.quantile(0.025)/1000000
    upper = values.quantile(0.975)/1000000
    mean = values.mean()/1000000
    risk = (values<0).mean()
    print(f'Средняя выручка: {mean:.2f} миллионов рублей')
    print(f'95% доверительный интервал от {lower:.2f} миллионов рублей до {upper:.2f} миллионов рублей')
    print(f'Риск убытка: {risk:.2f}')       

In [22]:
bootstrap(target0_valid, predictions0_valid)

Средняя выручка: 425.94 миллионов рублей
95% доверительный интервал от -102.09 миллионов рублей до 947.98 миллионов рублей
Риск убытка: 0.06


Рассчитаны показатели для первого региона

In [23]:
bootstrap(target1_valid, predictions1_valid)

Средняя выручка: 515.22 миллионов рублей
95% доверительный интервал от 68.87 миллионов рублей до 931.55 миллионов рублей
Риск убытка: 0.01


Рассчитаны показатели для второго региона

In [24]:
bootstrap(target2_valid, predictions2_valid)

Средняя выручка: 435.01 миллионов рублей
95% доверительный интервал от -128.88 миллионов рублей до 969.71 миллионов рублей
Риск убытка: 0.06


Рассчитаны показатели для третьего региона. Условию задачи о вероятности убытков менее 2,5% соответствует только второй регион. Это косвенно подтверждается RMSE, который для второго региона примерно в 50 раз меньше чем для первого и третьего. Соответственно, только он и может быть рекомендован к разработке